In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import dash
import dash_bootstrap_components as dbc
from dash import html,dcc,Input,Output,State
import plotly.figure_factory as ff
import plotly.express as px

# DataFrameの読み込み
parquetファイルにすると小さく保存できるする

In [101]:
# data = pd.read_csv('./data整形用/dashboard_data.csv',index_col=0)
data = pd.read_parquet('./data/dashboard_data.parquet')
data


,race_id,レース名,horse_id,馬名,着順,枠番,馬番,斤量,騎手,タイム,...,通過,ペース,上り,馬体重,勝ち馬(2着馬),賞金,surface,開催年,開催地,race_rank
554,201201010411,クイーンS(G3),2009106210,アイムユアーズ,1,5,8,52.0,池添謙一,1:47.2,...,5-5-5-4,35.8-35.1,34.7,468(+24),(ラブフール),3550.4,芝,2012,札幌,G3
555,201201010411,クイーンS(G3),2007103423,ラブフール,2,8,14,55.0,丸田恭介,1:47.2,...,14-14-13-12,35.8-35.1,33.7,472(-4),アイムユアーズ,1414.4,芝,2012,札幌,G3
556,201201010411,クイーンS(G3),2009102642,ミッドサマーフェア,3,6,10,51.0,蛯名正義,1:47.3,...,9-8-7-9,35.8-35.1,34.6,442(-10),アイムユアーズ,887.2,芝,2012,札幌,G3
557,201201010411,クイーンS(G3),2007104556,レインボーダリア,4,8,13,55.0,木幡初広,1:47.4,...,8-8-7-6,35.8-35.1,34.7,456(-8),アイムユアーズ,530.0,芝,2012,札幌,G3
558,201201010411,クイーンS(G3),2008100868,オールザットジャズ,5,2,2,55.0,藤岡佑介,1:47.4,...,5-7-7-6,35.8-35.1,34.7,450(+6),アイムユアーズ,350.0,芝,2012,札幌,G3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488105,202109021211,天皇賞(春)(G1),2016101209,オセアグレイト,13,8,15,58.0,横山典弘,3:17.5,...,17-17-14-11,35.8-37.4,39.0,496(0),ワールドプレミア,NaN,芝,2021,阪神,G1
488106,202109021211,天皇賞(春)(G1),2016102192,メイショウテンゲン,14,6,11,58.0,酒井学,3:18.1,...,9-8-10-14,35.8-37.4,39.4,470(+2),ワールドプレミア,NaN,芝,2021,阪神,G1
488107,202109021211,天皇賞(春)(G1),2016104291,ディバインフォース,15,4,8,58.0,武豊,3:18.1,...,16-16-17-17,35.8-37.4,39.0,460(-8),ワールドプレミア,NaN,芝,2021,阪神,G1
488108,202109021211,天皇賞(春)(G1),2014105947,シロニイ,16,2,4,58.0,松若風馬,3:18.4,...,4-5-5-7,35.8-37.4,40.3,502(-4),ワールドプレミア,NaN,芝,2021,阪神,G1


In [102]:
#race_nameドロップダウン用のリストを作成
race_name_main = data[['レース名']]
race_name_main['レース名']=race_name_main['レース名'].str[:-4]
race_name_main=race_name_main.drop_duplicates()['レース名']
#race_rankドロップダウン用のリストを作成
race_rank = data[['race_rank']].drop_duplicates()['race_rank']
race_rank=race_rank.sort_values()

In [103]:
race_name_main

554          クイーンS
1458          札幌記念
1608          エルムS
1769       キーンランドC
1930         札幌2歳S
            ...   
424074       小倉2歳S
460584    MBS賞スワンS
467117        優駿牝馬
467463        東京優駿
467497        目黒記念
Name: レース名, Length: 152, dtype: object

In [104]:
race_rank

14272    G1
1458     G2
554      G3
Name: race_rank, dtype: object

In [105]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20115 entries, 554 to 488109
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   race_id     20115 non-null  int64  
 1   レース名        20115 non-null  object 
 2   horse_id    20115 non-null  int64  
 3   馬名          20115 non-null  object 
 4   着順          20115 non-null  int64  
 5   枠番          20115 non-null  int64  
 6   馬番          20115 non-null  int64  
 7   斤量          20115 non-null  float64
 8   騎手          20115 non-null  object 
 9   タイム         19923 non-null  object 
 10  着差          18564 non-null  object 
 11  単勝          20115 non-null  object 
 12  人気          20045 non-null  float64
 13  調教師         20115 non-null  object 
 14  性別          20115 non-null  object 
 15  年齢          20115 non-null  int64  
 16  jockey_id   20115 non-null  int64  
 17  trainer_id  20115 non-null  int64  
 18  time        19923 non-null  float64
 19  体重          20115 non-null 

# Dash画面の構築

In [106]:
#dashアプリケーションの初期化
app = dash.Dash(external_stylesheets=[dbc.themes.CYBORG])

#ドロップダウン作成
#G1,G2,G3選択用
race_rank_options = [{'label':label,'value':value} for label,value in zip(race_rank,race_rank) ]
#レース名選択用(dropdownに与える値を50以上にするとかなり重くなり、152を与えたら動かなかった)
#そのためデータをG1,G2,G3で選択されたものから選ぶ様にする
#race_picker_options = [{'label':race_name_main,'value':race_name_main} for label,value in zip(race_name_main,race_name_main)]
#データ選択
describe_picker_options =[{'label':'1着馬','value':1},
                          {'label':'2着馬','value':2},
                          {'label':'3着馬','value':3},
                          {'label':'入着馬','value':'wide'},
                          {'label':'平均タイム','value':'mean'}]




sidebar = html.Div(
    [
        dbc.Row(
            [
                html.H5('Settings',
                        #marginで見出しの余白を指定できる
                        style={'margin-top': '5px', 'margin-left':'24px'})
                ],
            #styleで大きさ(viewportのv vhと書けば高さの比率で高さを指定できる)
            style={'height':'5vh'},
            #classNameに値を入れることで、背景色や文字色を変更できる bg-primary text-white
            className='bg-primary text-white font-italic'),
        dbc.Row(
            [
                #ドロップダウンの作成
                html.Div([
                    #レースランクを選択するドロップダウンの作成
                    html.P('Race Rank',
                           style={'margin-top':'8px','margin-bottom':'4px'},
                           className='font-weight-bold'),
                    dcc.Dropdown(id='race-rank-picker',
                                 multi=False,
                                 value='G1',
                                 options=race_rank_options,
                                 placeholder='Select a race rank',
                                 style={'width':'250px'}),
                    #レース名を選択するドロップダウンの作成
                    #タイトル部分
                    html.P('Race Name',
                           style={'margin-top':'8px','margin-bottom':'4px'},
                           className='font-weight-bold'),
                    #ドロップダウン部分(ここで選んだ値を、df.queryに与えてレースデータを絞る)
                    dcc.Dropdown(id='race-name-picker',
                                 multi=False,
                                 value='皐月賞(G1)',
                                #options=race_picker_options,
                                 placeholder='Select a race',
                                 style={'width':'250px'}
                                 ),
                    #平均、優勝馬などのデータを選択できる様にする(折れ線グラフ用)
                    html.P('Data Select',
                           style={'margin-top':'16px','margin-bottom':'4px'},
                           className='font-weight-bold'),
                    dcc.Dropdown(id='describe-picker',
                                 multi=False,
                                 value=1,
                                 options=describe_picker_options,
                                 placeholder='Select a data',
                                 style={'width':'250px'}),
                    #更新用ボタンの作成
                    html.Button(id='apply_button',
                                n_clicks=0,
                                children='apply',
                                style={'width':'100px',
                                       'margin-top':'16px'},
                                className='bg-warning text-white'),
                    html.Hr()
                    ]
                    )
            ],
        style ={'height':'45vh','margin-bottom':'0px'}),
    dbc.Row(
        [
            html.Div([
                html.P('Sex Ratio',className='font-weight-bold'),
                dcc.Graph(id='sex-ratio',
                          style={'height':'40vh'})
            ])
        ],
        style={'height':'50vh','margin':'8px'}
        )
    ]
)

content = html.Div(
    [
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Div([
                            html.P('Time Transition'),
                            dcc.Graph(id='fig-scatter',
                                      style={'height':'40vh'})
                            ])

                    ])
            ],
            style={'height':'50vh'}
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Div([
                            html.P('Jockey Results'),
                            dcc.Graph(id='jockey-ratio',
                                      style={'height':'45vh'},
                                      className='bg-white')])
                        ]),
                dbc.Col(
                    [
                        html.Div([
                            html.P('Age Ratio'),
                            dcc.Graph(id='age-ratio',
                                      style={'height':'45vh'},
                                      className='bg-white')])
                    ])
            ],
            style={'height':'50vh'})
        
    ]
)
    

#Contaierdで全体を定義
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                #sidberを全体の1/4を占める形に
                dbc.Col(sidebar,width=3,className='bg-light'),
                #contentを全体の3/4を占める形にする
                dbc.Col(content,width=9)
                ]
                ),
    ],
    fluid=True
)

#race-rankに応じたrace-nameを表示するcallback
@app.callback(
    Output('race-name-picker','options'),
    Input('race-rank-picker','value')
)
def update_dropdown_options(race_rank):
    #race-rank-pickerで選択されたデータに該当するドロップダウンのみ表示する
    rank_race_name= data.query('レース名.str.contains(@race_rank)')[['レース名']].drop_duplicates()['レース名']
    options = [{'label':label,'value':value} for label,value in zip(rank_race_name,rank_race_name)]
    return options


#各グラフの生成用のcallback
@app.callback(
    Output('sex-ratio','figure'),
    Output('fig-scatter','figure'),
    Output('jockey-ratio','figure'),
    Output('age-ratio','figure'),
    Input('apply_button','n_clicks'),
    [State('race-name-picker','value'),
    State('describe-picker','value')]
    )
def update_graph(n_clicks,race_name,describe):
    #race_nameから特定のデータだけを抽出
    race_data = data.query('レース名 == @race_name')
    #table_dataの作成(スペースの都合上縦書きの表にしたいのでtranspose)
    #table_data = race_data[['開催地','surface','distance']].mode().transpose()

    if describe in [1,2,3]:
        main_data = race_data.query('着順==@describe')
        sex_data = main_data['性別'].value_counts()
        jockey_data = main_data['騎手'].value_counts()
        age_data = main_data['年齢'].value_counts()
        #scatterのhoverがdescribeごとに異なるのでscatterplotごと分けて記載(主に着順のものには馬名をつけたい)
        fig_scatter = px.line(main_data,
                 x='開催年',
                 y='time',
                 hover_name='馬名',
                 hover_data=['馬場','天気','開催地','距離'],
                 markers=True
                 )
        
        
        

    elif describe == 'wide':
        main_data = race_data.query('着順 in [1,2,3]')
        sex_data = main_data['性別'].value_counts()
        jockey_data = main_data['騎手'].value_counts()
        age_data = main_data['年齢'].value_counts()
        #入着データなので、3着のtimeをplotしておく
        scatter_data = race_data.query('着順==3')
        fig_scatter = px.line(scatter_data,
                 x='開催年',
                 y='time',
                 hover_data=['馬場','天気','開催地','距離'],
                 markers=True
                 )

        
        
    elif describe == 'mean':
        sex_data=race_data['性別'].value_counts()
        jockey_data = race_data['騎手'].value_counts()
        age_data = race_data['年齢'].value_counts()
        describe_data = race_data[['race_id','time','開催年']].groupby(['race_id','開催年']).describe().reset_index()
        #describe_dataが2段カラムのため、天気データなどをmergeできないため1段に直す
        race_id=describe_data['race_id']
        race_year = describe_data['開催年']
        describe_data.drop(columns=['race_id','開催年'],inplace=True)
        level0=describe_data.columns.get_level_values(0)
        level1=describe_data.columns.get_level_values(1)
        describe_data.columns = level0 + '_' +level1
        describe_data['race_id']=race_id
        describe_data['開催年']=race_year
        race_details_data = race_data[['race_id','馬場','天気','開催地','距離','タイム']]
        #天気データをmerge(plot時に距離や馬場状態などのデータを表示できる様に)
        df_join = pd.merge(describe_data,race_details_data,on='race_id',how='left')
        df_join
        fig_scatter = px.line(df_join,
                 x='開催年',
                 y='time_mean',
                 hover_data=['馬場','天気','開催地','距離'],
                 markers=True
                 )
        
        
        
    #性別棒グラフの作成
    sex_ratio = px.bar(sex_data)
    #騎手棒グラフの作成
    jockey_ratio = px.bar(jockey_data)
    #年齢棒グラフの作成
    age_ratio = px.bar(age_data)

    return sex_ratio,fig_scatter,jockey_ratio,age_ratio
                            


In [107]:
#起動
if __name__ == "__main__":
    app.run_server(port=1234, debug=True,use_reloader=True)

/var/folders/xd/g3hbkn5d6rzfnjtxlkb8bt840000gn/T/ipykernel_12076/2744552233.py:219: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



http://localhost:1234/

